In [ ]:
!git clone https://github.com/lyexover/Projet-ITI

Cloning into 'Projet-ITI'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 13 (delta 0), reused 13 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (13/13), 13.05 MiB | 5.93 MiB/s, done.


In [ ]:
%cd Projet-ITI

/content/Projet-ITI


In [ ]:
!pip install opencv-python tqdm

# 2. Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import os
# Ton fichier JSON est dans ton repo cloné (ton_projet)
json_path = 'data/df_train_captions.json'

# Le chemin vers tes images sur Google Drive
# Assure-toi que 'images' est bien le nom du dossier qui contient tes .jpg
images_folder = '/content/drive/MyDrive/Colab Notebooks/Projet-ITI/train2017'

In [10]:
import cv2
import json
import numpy as np
from tqdm import tqdm

def extract_features(json_file, img_folder):
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Création du détecteur ORB
    # Nous utilisons 1000 features max par image pour un bon équilibre
    orb = cv2.ORB_create(nfeatures=1000)

    # Listes pour stocker les résultats
    all_descriptors_list = []
    image_features = {}

    print(f"Début de l'extraction de {len(data)} images...")

    for entry in tqdm(data):
        filename = entry['filename']
        img_id = entry['image_id']
        img_path = os.path.join(img_folder, filename)

        # Lecture et traitement de l'image
        img = cv2.imread(img_path)

        if img is None:
            # Utile pour ignorer les fichiers manquants ou endommagés
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Détection des points clés et calcul des descripteurs
        keypoints, descriptors = orb.detectAndCompute(gray, None)

        if descriptors is not None:
            # Stockage des descripteurs de cette image
            all_descriptors_list.append(descriptors)
            # Utilisation de np.float32 pour la compatibilité avec K-Means (Tâche 3)
            image_features[img_id] = descriptors.astype(np.float32)

    print("Extraction terminée !")

    # Concaténer tous les descripteurs en une seule grande matrice (nécessaire pour K-Means)
    if len(all_descriptors_list) > 0:
        all_descriptors_matrix = np.vstack(all_descriptors_list)
        # S'assurer que le type de données est correct pour K-Means (souvent requis par OpenCV)
        all_descriptors_matrix = all_descriptors_matrix.astype(np.float32)
    else:
        all_descriptors_matrix = None

    return all_descriptors_matrix, image_features

# --- EXECUTION ---
all_descs, img_feats = extract_features(json_path, images_folder)

if all_descs is not None:
    print(f"\n✅ Total des descripteurs extraits : {all_descs.shape[0]}")
    print(f"La matrice est prête pour la Tâche 3 (K-Means).")
else:
    print("\n❌ Aucun descripteur n'a pu être extrait. Vérifie tes chemins.")

Début de l'extraction de 5000 images...


100%|██████████| 5000/5000 [1:02:12<00:00,  1.34it/s]


Extraction terminée !

✅ Total des descripteurs extraits : 4882966
La matrice est prête pour la Tâche 3 (K-Means).


In [11]:
import numpy as np
import pickle
import os

# --- VÉRIFIEZ CE CHEMIN ---
# C'est le chemin de votre dossier 'Projet-ITI' sur Drive
# Il est important qu'il corresponde à votre structure de Drive
base_dir = '/content/drive/MyDrive/Colab Notebooks/Projet-ITI'
save_dir = os.path.join(base_dir, 'results')

# Création du dossier 'results' s'il n'existe pas
os.makedirs(save_dir, exist_ok=True)

# 1. Sauvegarde de la Matrice Globale (pour K-Means - Tâche 3)
# Utilisation du format .npy, très efficace pour stocker les arrays NumPy
# Matrice: (4882966, 32)
np.save(os.path.join(save_dir, 'all_descriptors_matrix.npy'), all_descs)
print(f"✅ Matrice globale sauvegardée pour la Tâche 3 : {os.path.join(save_dir, 'all_descriptors_matrix.npy')}")

# 2. Sauvegarde du Dictionnaire Par Image (pour Histogrammes - Tâche 4)
# Utilisation du format .pkl, idéal pour stocker les objets Python (dictionnaires)
with open(os.path.join(save_dir, 'image_features.pkl'), 'wb') as f:
    pickle.dump(img_feats, f)
print(f"✅ Dictionnaire des descripteurs par image sauvegardé pour la Tâche 4 : {os.path.join(save_dir, 'image_features.pkl')}")

print("\n🎉 Sauvegardes terminées. Vous pouvez passer à la Tâche 3.")

✅ Matrice globale sauvegardée pour la Tâche 3 : /content/drive/MyDrive/Colab Notebooks/Projet-ITI/results/all_descriptors_matrix.npy
✅ Dictionnaire des descripteurs par image sauvegardé pour la Tâche 4 : /content/drive/MyDrive/Colab Notebooks/Projet-ITI/results/image_features.pkl

🎉 Sauvegardes terminées. Vous pouvez passer à la Tâche 3.
